<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Raimundo Sandoval
- Nombre de alumno 2: Cristóbal Subiabre


### **Link de repositorio de GitHub:** `http://https://github.com/raisando/Laboratorio-MDS7202-`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
import os
try:
    from google.colab import drive
    drive.mount("/content/drive")
    #path = '/content/drive/MyDrive/U. Chile/9no Semestre 2023/Laboratorio Programación Cientifica/Lab_8'
    path = '/content/drive/MyDrive/U/Laboratorio Programación Cientifica/Lab_8'
    os.chdir(path)
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [ ]:
df_players['labels'] = df_players['National_Position'].fillna(0)
df_players['labels'] = df_players['labels'].apply(lambda x: 1 if x != 0 else x)
df_players['labels'].value_counts()

0    16513
1     1075
Name: labels, dtype: int64

De lo anterior es importante notar que hay muchas entradas las cuales eran NA, que corresponde segun se menciono a jugadores no seleccionados, por ello, se asigna dicha categoria a los NA. Es decir, 0 = No Seleccionado y 1 = Seleccionado.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [ ]:
ct= ColumnTransformer([
    ("StandardScaler",StandardScaler(),["Height","Weight","Aggression","Reactions","Vision","Composure","Crossing","Short_Pass","Long_Pass","Acceleration","Speed","Stamina","Strength","Balance","Agility","Jumping","Freekick_Accuracy"]),
    ("PowerTransform",PowerTransformer(),['Age',"Ball_Control","Dribbling","Marking","Sliding_Tackle","Standing_Tackle","Interceptions","Heading","Shot_Power"]),
    ("MinMaxScaler",MinMaxScaler(),["Finishing","Long_Shots","Curve","Penalties","Volleys"]),
    ("OneHot",OneHotEncoder(handle_unknown="ignore"),["Preffered_Foot","Work_Rate","Skill_Moves"])
])

In [ ]:
transformations = [
    ('numeric', MinMaxScaler(), ['Height', 'Weight', 'Age', 'Aggression', 'Reactions', 'Vision', 'Composure', 'Long_Pass', 'Acceleration', 'Speed', 'Strength', 'Balance', 'Agility', 'Jumping', 'Finishing', 'Volleys']),
    ('numeric_1', StandardScaler(), ['Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Interceptions', 'Crossing', 'Short_Pass', 'Stamina', 'Heading', 'Shot_Power', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties']),
    ('categorical', OneHotEncoder( handle_unknown='ignore'), ['Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate', 'Weak_foot', 'Skill_Moves' ]),
]

ct = ColumnTransformer(transformers=transformations)

**Respuesta:** Se selecciono MinMaxScaler en las columnas que tenian una distribucion uniforme y StandardScaler en aquellas que no se podian describir segun una distribución uniforme. Por último, se uso OneHotEncoder en aquellas donde las variables eran categoricas

**Respuesta:** Tambien, se seleccionaron los modelos de XBG, RF y KNN, los dos primeros corresponden a modelos de ensamblaje, y el otro un Kneighbors,esto porque como muestra la guia, estamos buscando predecir una categoria con menos de 100000 muestras y solo columnas numericas.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

xgb_pipe = Pipeline(
    [("preprocesamiento", ct), ('classifier', xgb.XGBClassifier())]
)

rf_pipe = Pipeline(
    [("preprocesamiento", ct), ('classifier', RandomForestClassifier())]
)

knn_pipe = Pipeline(
    [("preprocesamiento", ct), ('classifier', KNeighborsClassifier(n_neighbors=3))]
)


### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [ ]:
X = df_players.drop('labels', axis = 1)
y = df_players['labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1999)

In [ ]:
xgb_pipe.fit(X_train, y_train)
rf_pipe.fit(X_train, y_train)
knn_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('numeric', MinMaxScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Aggression', 'Reactions',
                                                   'Vision', 'Composure',
                                                   'Long_Pass', 'Acceleration',
                                                   'Speed', 'Strength',
                                                   'Balance', 'Agility',
                                                   'Jumping', 'Finishing',
                                                   'Volleys']),
                                                 ('numeric_1', StandardScaler(),
                                                  ['Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Interceptions', 'Crossing',
                                                   'Short_Pass', 'Stamina',
                                                   'Heading', 'Shot_Power',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Nationality',
                                                   'Club_Position',
                                                   'Preffered_Foot',
                                                   'Work_Rate', 'Weak_foot',
                                                   'Skill_Moves'])])),
                ('classifier', KNeighborsClassifier(n_neighbors=3))])

In [ ]:
y_pred_tree = xgb_pipe.predict(X_test)
y_pred_rf = rf_pipe.predict(X_test)
y_pred_knn = knn_pipe.predict(X_test)

In [ ]:
print('Reporte clasifiacion de pipeline con XGB classifier')
print(classification_report(y_test, y_pred_tree))
print('------------------------------------------')

print('Reporte clasifiacion de pipeline con Random Forest')
print(classification_report(y_test, y_pred_rf))
print('------------------------------------------')

print('Reporte clasifiacion de pipeline con KNN')
print(classification_report(y_test, y_pred_knn))
print('------------------------------------------')

Reporte clasifiacion de pipeline con XGB classifier
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      3301
           1       0.67      0.38      0.49       217

    accuracy                           0.95      3518
   macro avg       0.82      0.69      0.73      3518
weighted avg       0.94      0.95      0.94      3518

------------------------------------------
Reporte clasifiacion de pipeline con Random Forest
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3301
           1       0.69      0.10      0.18       217

    accuracy                           0.94      3518
   macro avg       0.82      0.55      0.57      3518
weighted avg       0.93      0.94      0.92      3518

------------------------------------------
Reporte clasifiacion de pipeline con KNN
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      3301
       

**Respuesta**:
Accuracy habla sobre la cantidad de predicciones correctas. En general no diferencia entre accuracy para clase positiva o negativa, si no que engloba ambas predicciones y representa cuantas labels estuvieron bien puestas. En el caso de, por ejemplo, el modelo de XGB: un 95% de los jugadores fue correctamente clasificado ya sea como seleccionado nacional o no seleccionado nacional. 

La Precision si algo mas directo sobre la clase positiva. Rrefiriendonos solamente a la clase positiva = 1 = seleccionado nacional, se tiene que la precision corresponde a la proporcion de los jugadores predichos como seleccionados, que efectivamente son seleccionados nacionales. Asimismo, la recall nos dice que porcentaje de los seleccionados nacionales fue correctamente predicho como seleccionado por el modelo.

**Respuesta:** En cuanto a los resultados, en general podria se posible mejorarlos mediante tecnicas mas ajustadas de preprocesamiento de datos, o creando features extras al modelo. Tambien, es claro que el modelo no cuenta con suficientes muestras de la clase positiva para clasificar correctamente. Solucionando este desbalance los resultados sin duda mejoraran, pues en genaral un desbalance asi de grande (superior a 1:10) afecta la capacidad de generalizacion del modelo y, aun usando modelos de ensamblaje,sobreajustandose a la clase mayoritaria.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [ ]:
# Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
df2 = df_players.copy()
df2["Club_Position"] = df2["Club_Position"].map({'ST': "ataque", 'CF': "ataque",
                                'RW': "central_ataque", 'CAM': "central_ataque", 'LW': "central_ataque", 
                                'RM': "central", 'CM': "central", 'LM': "central", 
                                'RWB': "central_defensa", 'CDM': "central_defensa", 'LWB': "central_defensa",
                                'RB': "defensa", 'CB': "defensa", 'LB': "defensa",
                                'GK': "arquero"})
df2 = df2.dropna(subset="Club_Position", axis=0)

In [ ]:
#  Cuente cuantos por clase quedan
df2["Club_Position"].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64

In [ ]:
y = df2["Club_Position"].to_list()
X = df2.drop(columns = ["Name", "Nationality", "National_Position", "Club_Position"])

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_enc = label_encoder.fit_transform(y)
y_enc

array([3, 3, 3, ..., 2, 5, 5])

In [ ]:
ct2= ColumnTransformer([
    ("StandardScaler",StandardScaler(),["Height","Weight","Aggression","Reactions","Vision","Composure","Crossing","Short_Pass","Long_Pass","Acceleration","Speed","Stamina","Strength","Balance","Agility","Jumping","Freekick_Accuracy"]),
    ("PowerTransform",PowerTransformer(),['Age',"Ball_Control","Dribbling","Marking","Sliding_Tackle","Standing_Tackle","Interceptions","Heading","Shot_Power"]),
    ("MinMaxScaler",MinMaxScaler(),["Finishing","Long_Shots","Curve","Penalties","Volleys"]),
    ("OneHot",OneHotEncoder(handle_unknown="ignore"),["Preffered_Foot","Work_Rate","Skill_Moves"])
])

pipe2 = Pipeline([("ColumnTransform", ct2),
                     ('classifier', RandomForestClassifier())])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Entrenamiento y predicción
pipe2.fit(X_train, y_train)
y_pred2 = pipe2.predict(X_test)
print(classification_report(y_test, y_pred2))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       177
         ataque       0.82      0.82      0.82       138
        central       0.59      0.63      0.61       287
 central_ataque       0.49      0.33      0.39       163
central_defensa       0.48      0.18      0.26        62
        defensa       0.80      0.98      0.88       355

       accuracy                           0.75      1182
      macro avg       0.70      0.65      0.66      1182
   weighted avg       0.72      0.75      0.73      1182



**Respuesta**: Se ve que hay algunas clases que tienen mejor performance que otras. Esto nuevamnte se puede atribuir al desbalance de las clases entre ellas. para mejorar, primer deseable seria tener un a distribucion lo mas equitativa posible entre las clases, pero aun asi vemos que las clases con mas datos no necesariamente tienen buena performance, esto nos guia a pensar que quizas las features con las que se cuenta actualmente no son suficientes para diferenciar todas las posiciones, y sin hablar tampoco de que existe una cantidad de jugadores con habilidades suficientes para jugar en distintas posiciones.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta** La métrica $R^2$ representa la proporción de varianza de los valores predichos por el modelo con respecto a la las etiquetas reales. dada esa definicion, el mejor puntaje posible es 1 y si el modelo es suficientemente malo puede incluso tener resultados negativos.

In [ ]:
salarios = pd.read_csv("salarios.csv")
salarios = salarios.drop("Unnamed: 0", 1)
salarios = salarios.rename(columns={"Player": "Name"})

<ipython-input-21-29d69209a54d>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  salarios = salarios.drop("Unnamed: 0", 1)


In [ ]:
df3 = pd.merge(left=df_players,
                    right=salarios,
                    on="Name",
                    sort=True,
                    how="outer")
df3 = df3.dropna(subset="Weekly Salary", axis=0)
df3 = df3.dropna(subset="Height", axis=0)

In [ ]:
ct3= ColumnTransformer([
    ("StandardScaler",StandardScaler(),["Height","Weight","Aggression","Reactions","Vision","Composure","Crossing","Short_Pass","Long_Pass","Acceleration","Speed","Stamina","Strength","Balance","Agility","Jumping","Freekick_Accuracy"]),
    ("PowerTransform",PowerTransformer(),['Age',"Ball_Control","Dribbling","Marking","Sliding_Tackle","Standing_Tackle","Interceptions","Heading","Shot_Power"]),
    ("MinMaxScaler",MinMaxScaler(),["Finishing","Long_Shots","Curve","Penalties","Volleys"]),
    ("OneHot",OneHotEncoder(handle_unknown="ignore"),["Nationality","Preffered_Foot","Work_Rate","Skill_Moves"])
])

In [ ]:
y = df3["Weekly Salary"].to_list()
X = df3.drop(["Name", "Weekly Salary"], 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

<ipython-input-24-8efb75f7c98e>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df3.drop(["Name", "Weekly Salary"], 1)


**Respuesta** Como en este caso queremos predecir un numero, seguimos teniendo menos de 100000 datos y creemos que algunas pocas variables deben ser importantes para el sueldo, se uso un regressor LASSO, sin embargo este al no tener la performance deseada se paso a un regressor de ElasticNet

In [ ]:
#from sklearn import linear_model
from sklearn.linear_model import ElasticNet

elastic_pipe =Pipeline([('ColumnTransform', ct3) , 
                       ('Regresor', ElasticNet(alpha=0.01))])

In [ ]:
from sklearn.metrics import r2_score

elastic_pipe.fit(X_train, y_train)

elastic_y_pred = elastic_pipe.predict(X_test)

print(f" R² score: {r2_score(y_test, elastic_y_pred)}")         

 R² score: 0.36680824803810697


**Respuesta**: En general la performance no es buena, es de hecho solo un poco mejor de lo pedido. Se debe tomar en cuenta que en si la cantidad de filas no es tanta, y particularmente las caracteristicas que se presentan no parecieran muy intuitivas para calcular el sueldo de un jugador. Por lo tanto para aumentar la performance del modelo se podria comenzar a trabajar en features que si den informacion relevante para este tipo de calculos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>